# S09 Lab Exercise 

## Víctor Vega Sobral

### Explanations 
The attached files are a collection of tweets labelled with sentiment in 3 categories:

```json
sentiments = {
    "LABEL_0": "Bearish", 
    "LABEL_1": "Bullish", 
    "LABEL_2": "Neutral"
}  
```

1. Train a LSTM network to with the training file. 

2. Validate the trained model with the valid file. 

3. Comment what you are doing in each part of your code. As the better the code, comments and result validation as the better the grade.

Remember that you have to send the final file in this exercise and the file must be in your digital portfolio with all the proper commits done.

- ``sent_train.csv`` 
- ``sent_valid.csv`` 



In [ ]:
__author__ = "Victor Vega Sobral"